In [1]:
#Import libraries
from bs4 import BeautifulSoup
import time
import urllib.request
import pandas as pd
import os

In [2]:
# Create a directory if it does not exist
if not os.path.exists('fanfics'):
    os.mkdir('fanfics')

# Loop through the 100 pages of fanfiction and save HTML files to "fanfics" folder
for x in range(1, 5000, 20):
    url = f'https://archiveofourown.org/tags/%E5%83%95%E3%81%AE%E3%83%92%E3%83%BC%E3%83%AD%E3%83%BC%E3%82%A2%E3%82%AB%E3%83%87%E3%83%9F%E3%82%A2%20|%20Boku%20no%20Hero%20Academia%20|%20My%20Hero%20Academia%20(Anime%20*a*%20Manga)/works?page={x}'
    try:
        with urllib.request.urlopen(url) as response:
            html = response.read()
            html = html.decode('utf-8')

        with open (f'fanfics/AO3_Page{x}_spread.html', 'w', encoding="utf-8") as new_file:
            new_file.write(html)
        time.sleep(8)
    except urllib.error.HTTPError as e:
        if e.code == 429:
                print('Too many requests!---SLEEPING---')
                print('we should restart on page', x)
                print('we should restart with this url:', url)
                break
        raise
        

In [3]:
#Creating soup objects and putting them in a list
AO3_list = []
for i in range(1, 5000, 20):
    html = f'fanfics/AO3_Page{i}_spread.html'
    ao3_soup = BeautifulSoup(open(html,encoding='utf-8'), 'html.parser')
    AO3_list.append(ao3_soup)


In [4]:
url = 'https://archiveofourown.org'

In [5]:
def getWebsiteAsSoup(url):

    req = urllib.request.Request(url)
    with urllib.request.urlopen(req) as response:
        story_html = response.read()
    
    story_soup = BeautifulSoup(story_html, 'html.parser')
    with open('story_spread.html', 'w') as new_file:
        new_file.write(str(story_soup))
        
    return story_soup

In [6]:
stories = []
fanfics = []
relationship = []
character = []
mix_fandom = []

for x in range(len(AO3_list)):
    for titles in AO3_list[x].find_all('li', role = 'article'):
        fanfic = {}
        title = titles.find('a').get_text()
        fanfic['title'] = title
        
        url_tail = titles.find('a').get('href')
        link = url + url_tail
        stories.append([title, link])
        
        fandom = titles.find_all('h5', class_ = 'fandoms heading')
        fandoms = [tag.get_text().strip() for tag in fandom]
        fanfic['mixed fandom'] = fandoms
        mix_fandom.append(fandoms)
        
        relationships = titles.find_all('li', class_ = 'relationships')
        ships = [tag.get_text().strip() for tag in relationships]
        fanfic['ships'] = ships
        
        char = titles.find_all('li', class_ = 'characters')
        characters = [tag.get_text() for tag in char]
        fanfic['characters'] = (characters)
        
        try:
            hits = titles.find('dd', 'hits').get_text()
        except:
            hits = None
        fanfic['hits'] = hits
        
        try:
            kudos = titles.find('dd', 'kudos').get_text()
        except:
            kudos = None
        fanfic['kudos'] = kudos
        
        try:
            fic_soup = getWebsiteAsSoup(link)
            fic_soup.prettify()
            for info in fic_soup:
                published = info.find('dd', class_ = 'published').get_text()
                fanfic['published'] = published
                relationship.append([ships, published])
                character.append([characters, published])
        except:
            published = titles.find('p', class_ = 'datetime').get_text()
            fanfic['published'] = published
            character.append([characters, published])
            relationship.append([ships, published])
        fanfics.append(fanfic)

In [7]:
fanfics_df = pd.DataFrame.from_dict(fanfics, orient = 'columns')
fanfics_df['published'].value_counts()

21 Apr 2023    20
31 May 2022    20
27 Jul 2022    20
19 Jul 2022    20
20 Apr 2023    20
               ..
13 Feb 2022     1
07 Feb 2022     1
08 Dec 2022     1
18 Mar 2023     1
13 Dec 2022     1
Name: published, Length: 328, dtype: int64

In [8]:
fanfics_df.to_csv("AO3_spread.csv", index=False)

In [9]:
shipings = pd.DataFrame(relationship)
shipings.columns = ['ships', 'dates']
ships = shipings[219:]
ships = ships.explode('ships')

In [10]:
relationship.sort()

In [11]:
ships.to_csv("shipsviadate_spread.csv", index=False)

In [12]:
characters = pd.DataFrame(character)
characters.columns = ['names', 'dates']
characters = characters[65:]
characters = characters.explode('names')

In [13]:
character.sort()

In [14]:
characters.to_csv('charactersviadate_spread.csv', index = False)